# Data Wrangling

### 0.0 Data Collection

In [8]:
# Import libraries
import glob
import os

import pandas as pd

In [41]:
# Collect paths
book_train_path = './Data/book_train.parquet'
book_test_path = './Data/book_test.parquet/'
trade_train_path = './Data/trade_train.parquet/'
trade_test_path = './Data/trade_test.parquet/'

In [42]:
# Read in data
book_train = pd.read_parquet(book_train_path)
book_test = pd.read_parquet(book_test_path)
trade_train = pd.read_parquet(trade_train_path)
trade_test = pd.read_parquet(book_train_path)

In [43]:
trade_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167253289 entries, 0 to 167253288
Data columns (total 11 columns):
 #   Column             Dtype   
---  ------             -----   
 0   time_id            int16   
 1   seconds_in_bucket  int16   
 2   bid_price1         float32 
 3   ask_price1         float32 
 4   bid_price2         float32 
 5   ask_price2         float32 
 6   bid_size1          int32   
 7   ask_size1          int32   
 8   bid_size2          int32   
 9   ask_size2          int32   
 10  stock_id           category
dtypes: category(1), float32(4), int16(2), int32(4)
memory usage: 5.8 GB


In [44]:
trade_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11395007 entries, 0 to 11395006
Data columns (total 6 columns):
 #   Column             Dtype   
---  ------             -----   
 0   time_id            int16   
 1   seconds_in_bucket  int16   
 2   price              float32 
 3   size               int32   
 4   order_count        int16   
 5   stock_id           category
dtypes: category(1), float32(1), int16(3), int32(1)
memory usage: 163.0 MB


In [45]:
book_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   time_id            3 non-null      int16   
 1   seconds_in_bucket  3 non-null      int16   
 2   bid_price1         3 non-null      float32 
 3   ask_price1         3 non-null      float32 
 4   bid_price2         3 non-null      float32 
 5   ask_price2         3 non-null      float32 
 6   bid_size1          3 non-null      int32   
 7   ask_size1          3 non-null      int32   
 8   bid_size2          3 non-null      int32   
 9   ask_size2          3 non-null      int32   
 10  stock_id           3 non-null      category
dtypes: category(1), float32(4), int16(2), int32(4)
memory usage: 355.0 bytes


In [46]:
# Note that by convention at Kaggle the book_test file only has 3 entries.
book_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167253289 entries, 0 to 167253288
Data columns (total 11 columns):
 #   Column             Dtype   
---  ------             -----   
 0   time_id            int16   
 1   seconds_in_bucket  int16   
 2   bid_price1         float32 
 3   ask_price1         float32 
 4   bid_price2         float32 
 5   ask_price2         float32 
 6   bid_size1          int32   
 7   ask_size1          int32   
 8   bid_size2          int32   
 9   ask_size2          int32   
 10  stock_id           category
dtypes: category(1), float32(4), int16(2), int32(4)
memory usage: 5.8 GB
